# Word Embeddings

Apply functions from the practical classes notebooks, in order to compare the results with the baseline classification

## Word2Vec in Gensim

[Word2Vec](https://code.google.com/archive/p/word2vec/) is a model for training word embeddings that revolutionized the way words are represented. [Gensim](https://radimrehurek.com/gensim_3.8.3/models/word2vec.html) provides an implementation of the algorithm, with which we can train our own word embeddings.

In [1]:
from gensim.models import Word2Vec, KeyedVectors

## Article's body

Our first task is to create a model based on the article's body, hoping that it is easier to predict the labels on a corpus with the same thematics

In [2]:
import pandas as pd

articles = pd.read_excel("OpArticles.xlsx")

In [3]:
import re

documents = []
for i in range(0, articles['body'].size):
    # get review, remove non alpha chars and convert to lower-case
    review = re.sub('[^a-zA-Z\u00C0-\u00ff]', ' ', articles['body'][i]).lower()
    # add review to corpus
    documents.append(review.split())

#### Training the Word2Vec model

In [4]:
from time import time

start_time = time()
model_articles = Word2Vec(documents, vector_size=150, window=10, min_count=2, workers=10, sg=1)

print("Training time:", time() - start_time)

Training time: 2.9753782749176025


In [5]:
model_articles.wv.save("./word_vectors/wv_articles")

In [6]:
model_articles = KeyedVectors.load("./word_vectors/wv_articles")

## Portuguese embeddings

A number of embeddings for Portuguese are available at [NILC](http://nilc.icmc.usp.br/embeddings), as well as at the [NLX-group](https://github.com/nlx-group/LX-DSemVectors).

We will be testing the following models from NILC
- Word2Vec CBOW 100 dimensions
- Word2Vec SKIP-GRAM 100 dimensions
- FastText CBOW 100 dimensions
- FastText SKIP-GRAM 100 dimensions
- FastText SKIP-GRAM 1000 dimensions

Only need to load from .txt once, then load only created word vectors

In [7]:
# takes a while to load...
model_w2v_cbow_s100 = KeyedVectors.load_word2vec_format('./word_vectors/w2v_cbow_s100.txt')
# save model word vectors
model_w2v_cbow_s100.save("./word_vectors/wv_w2v_cbow_s100")

In [8]:
model_w2v_skip_s100 = KeyedVectors.load_word2vec_format('./word_vectors/w2v_skip_s100.txt')
model_w2v_skip_s100.save("./word_vectors/wv_w2v_skip_s100")

In [9]:
model_ft_cbow_s100 = KeyedVectors.load_word2vec_format('./word_vectors/ft_cbow_s100.txt')
model_ft_cbow_s100.save("./word_vectors/wv_ft_cbow_s100")

In [10]:
model_ft_skip_s100 = KeyedVectors.load_word2vec_format('./word_vectors/ft_skip_s100.txt')
model_ft_skip_s100.save("./word_vectors/wv_ft_skip_s100")

In [11]:
model_ft_skip_s1000 = KeyedVectors.load_word2vec_format('./word_vectors/ft_skip_s1000.txt')
model_ft_skip_s1000.save("./word_vectors/wv_ft_skip_s1000")

Load model word vectors (much faster than the above)

In [12]:
model_w2v_cbow_s100 = KeyedVectors.load("./word_vectors/wv_w2v_cbow_s100")

In [13]:
model_w2v_skip_s100 = KeyedVectors.load("./word_vectors/wv_w2v_skip_s100")

In [14]:
model_ft_cbow_s100 = KeyedVectors.load("./word_vectors/wv_ft_cbow_s100")

In [15]:
model_ft_skip_s100 = KeyedVectors.load("./word_vectors/wv_ft_skip_s100")

In [16]:
model_ft_skip_s1000 = KeyedVectors.load("./word_vectors/wv_ft_skip_s1000")

## Load Dataset

In [17]:
dataset = pd.read_excel('OpArticles_ADUs.xlsx')

#### Cleanup

In [18]:
corpus = []
for i in range(0, dataset['tokens'].size):
    # get review, remove non alpha chars and convert to lower-case
    review = re.sub('[^a-zA-Z\u00C0-\u00ff]', ' ', dataset['tokens'][i]).lower()
    # add review to corpus
    corpus.append(review)

#### Fixing the length of the input

The reviews in our corpus have variable length. However, we need to represent them with a fixed-length vector of features. One way to do it is to impose a limit on the number of word embeddings we want to include.

To convert words into their vector representations (embeddings), let's create an auxiliary function that takes in the number of embeddings we wish to include in the representation:

In [19]:
import numpy as np

def text_to_vector(embeddings, text, sequence_len):
    
    # split text into tokens
    tokens = text.split()
    
    # convert tokens to embedding vectors, up to sequence_len tokens
    vec = []
    n = 0
    i = 0
    while i < len(tokens) and n < sequence_len:   # while there are tokens and did not reach desired sequence length
        try:
            vec.extend(embeddings.get_vector(tokens[i]))
            n += 1
        except KeyError:
            True   # simply ignore out-of-vocabulary tokens
        finally:
            i += 1
    
    # add blanks up to sequence_len, if needed
    for j in range(sequence_len - n):
        vec.extend(np.zeros(embeddings.vector_size,))
    
    return vec

The above *text_to_vector* function takes an *embeddings* dictionary, the *text* to convert, and the number of words *sequence_len* from *text* to consider. It returns a vector with appended embeddings for the first *sequence_len* words that exist in the *embeddings* dictionary (tokens for which no embedding is found are ignored). In case the text has less than *sequence_len* words for which we have embeddings, blank embeddings will be added.

To better decide how many word embeddings we wish to append, let's learn a bit more about the length of each review in our corpus.

In [20]:
from scipy import stats

lens = [len(c.split()) for c in corpus]
print(np.min(lens), np.max(lens), np.mean(lens), np.std(lens), stats.mode(lens))

1 82 14.30406737143881 9.470560303048728 ModeResult(mode=array([8]), count=array([972]))


## Predictions

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [22]:
def predict(model):
    # convert corpus into dataset with appended embeddings representation
    embeddings_corpus = []
    for c in corpus:
        embeddings_corpus.append(text_to_vector(model, c, 15))

    X = np.array(embeddings_corpus)
    y = dataset['label']

    print(X.shape, y.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, stratify=y)

    clf = SGDClassifier(random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification report:\n", classification_report(y_test, y_pred))

### Article's body model

In [23]:
predict(model_articles)

(16743, 2250) (16743,)

Confusion matrix:
 [[  83    3  623    8   16]
 [   4    5  120    2    2]
 [  61    2 1514   14   30]
 [   9    0  252   14    7]
 [  16    0  530   14   20]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.48      0.11      0.18       733
      Policy       0.50      0.04      0.07       133
       Value       0.50      0.93      0.65      1621
    Value(+)       0.27      0.05      0.08       282
    Value(-)       0.27      0.03      0.06       580

    accuracy                           0.49      3349
   macro avg       0.40      0.23      0.21      3349
weighted avg       0.43      0.49      0.38      3349



### NILC PT Models

In [24]:
predict(model_w2v_cbow_s100)

(16743, 1500) (16743,)

Confusion matrix:
 [[ 257    7  360   33   76]
 [   9   42   71    4    7]
 [ 235   38 1114   63  171]
 [  56    7  154   45   20]
 [  79   10  316   10  165]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.40      0.35      0.38       733
      Policy       0.40      0.32      0.35       133
       Value       0.55      0.69      0.61      1621
    Value(+)       0.29      0.16      0.21       282
    Value(-)       0.38      0.28      0.32       580

    accuracy                           0.48      3349
   macro avg       0.41      0.36      0.37      3349
weighted avg       0.46      0.48      0.47      3349



In [25]:
predict(model_w2v_skip_s100)

(16743, 1500) (16743,)

Confusion matrix:
 [[ 252    6  342   40   93]
 [   8   46   71    5    3]
 [ 231   39 1088   84  179]
 [  47   10  147   54   24]
 [  74   14  316   18  158]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.41      0.34      0.37       733
      Policy       0.40      0.35      0.37       133
       Value       0.55      0.67      0.61      1621
    Value(+)       0.27      0.19      0.22       282
    Value(-)       0.35      0.27      0.30       580

    accuracy                           0.48      3349
   macro avg       0.40      0.36      0.38      3349
weighted avg       0.46      0.48      0.46      3349



In [26]:
predict(model_ft_cbow_s100)

(16743, 1500) (16743,)

Confusion matrix:
 [[278  11 209  59 176]
 [ 15  53  38   8  19]
 [372  45 663 111 430]
 [ 54  10  87  77  54]
 [115  10 158  33 264]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.33      0.38      0.35       733
      Policy       0.41      0.40      0.40       133
       Value       0.57      0.41      0.48      1621
    Value(+)       0.27      0.27      0.27       282
    Value(-)       0.28      0.46      0.35       580

    accuracy                           0.40      3349
   macro avg       0.37      0.38      0.37      3349
weighted avg       0.44      0.40      0.41      3349



In [27]:
predict(model_ft_skip_s100)

(16743, 1500) (16743,)

Confusion matrix:
 [[277   1 306  67  82]
 [ 11  38  68  10   6]
 [226  23 993 136 243]
 [ 41   6 121  82  32]
 [ 65   1 271  20 223]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.45      0.38      0.41       733
      Policy       0.55      0.29      0.38       133
       Value       0.56      0.61      0.59      1621
    Value(+)       0.26      0.29      0.27       282
    Value(-)       0.38      0.38      0.38       580

    accuracy                           0.48      3349
   macro avg       0.44      0.39      0.41      3349
weighted avg       0.48      0.48      0.48      3349



In [28]:
predict(model_ft_skip_s1000)

(16743, 15000) (16743,)

Confusion matrix:
 [[301   1 293  40  98]
 [  3  60  47  14   9]
 [301  19 951  89 261]
 [ 46   3 105 106  22]
 [ 74   2 211   9 284]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.42      0.41      0.41       733
      Policy       0.71      0.45      0.55       133
       Value       0.59      0.59      0.59      1621
    Value(+)       0.41      0.38      0.39       282
    Value(-)       0.42      0.49      0.45       580

    accuracy                           0.51      3349
   macro avg       0.51      0.46      0.48      3349
weighted avg       0.51      0.51      0.51      3349

